In [245]:
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

from joblib import dump, load

import numpy as np

In [276]:
#loading saved model
regressor = load('20220818_randomforest_model.joblib')

In [281]:
# Read player model and ages
xl1 = pd.read_csv('data_cleaned/modelSource.csv', index_col=0)

In [282]:
xl1 = xl1.loc[xl1.season==2021]
xl1 = xl1.reset_index(drop=True)
xl1

,season,week,team,player,age,pos,posRank,opponent,passA,passC,...,defSack_prior1_opp,defI_prior1_opp,defSaf_prior1_opp,defFum_prior1_opp,defBlk_prior1_opp,defT_prior1_opp,defPtsAgainst_prior1_opp,defPassYAgainst_prior1_opp,defRushYAgainst_prior1_opp,defYdsAgainst_prior1_opp
0,2021.0,1.0,ARI,Kyler Murray,24.0,QB,QB1,TEN,32.0,21.0,...,1.1875,0.9375,0.0000,0.5000,0.0625,0.1250,27.4375,277.4375,120.8125,398.2500
1,2021.0,1.0,KCC,Patrick Mahomes,26.0,QB,QB1,CLE,36.0,27.0,...,2.3750,0.6875,0.1250,0.6250,0.1875,0.1250,26.1875,247.6250,110.8125,358.4375
2,2021.0,1.0,NOS,Jameis Winston,27.0,QB,QB1,GBP,20.0,14.0,...,2.5625,0.6875,0.0625,0.4375,0.0000,0.1250,23.0625,221.1875,112.8125,334.0000
3,2021.0,1.0,TBB,Tom Brady,44.0,QB,QB1,DAL,50.0,32.0,...,1.9375,0.6250,0.0000,0.8125,0.0625,0.1875,29.5625,227.6250,158.8125,386.4375
4,2021.0,1.0,SFO,Jared Goff,27.0,QB,QB1,DET,57.0,38.0,...,1.5000,0.4375,0.0625,0.3125,0.2500,0.0625,32.4375,284.8750,134.8750,419.7500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5753,2021.0,17.0,MIA,Miami DolphinsMIA,0.0,DF,DF1,TEN,0.0,0.0,...,1.1875,0.9375,0.0000,0.5000,0.0625,0.1250,27.4375,277.4375,120.8125,398.2500
5754,2021.0,17.0,DET,Detroit LionsDET,0.0,DF,DF1,SEA,0.0,0.0,...,2.8750,0.8750,0.1875,0.5000,0.1250,0.0000,23.1875,285.0000,95.5625,380.5625
5755,2021.0,17.0,DEN,Denver BroncosDEN,0.0,DF,DF1,LAC,0.0,0.0,...,1.6875,0.7500,0.0625,0.4375,0.1250,0.1250,26.6250,223.6250,119.8125,343.4375
5756,2021.0,17.0,MIN,Minnesota VikingsMIN,0.0,DF,DF1,GBP,0.0,0.0,...,2.5625,0.6875,0.0625,0.4375,0.0000,0.1250,23.0625,221.1875,112.8125,334.0000


In [283]:
# Define features and labels
y = xl1[[
'passA',
'passC',
'passY',
'passT',
'passI',
'pass2',
'rushA',
'rushY',
'rushT',
'rush2',
'recC',
'recY',
'recT',
'rec2',
'fum',
'XPA',
'XPM',
'FGA',
'FGM',
'FG50',
'defSack',
'defI',
'defSaf',
'defFum',
'defBlk',
'defT',
'defPtsAgainst',
'defPassYAgainst',
'defRushYAgainst',
'defYdsAgainst'                                 
]]

X = xl1.drop(columns=[
    'season',
    'team',
    'player',
    'opponent',
    'passA',
    'passC',
    'passY',
    'passT',
    'passI',
    'pass2',
    'rushA',
    'rushY',
    'rushT',
    'rush2',
    'recC',
    'recY',
    'recT',
    'rec2',
    'fum',
    'XPA',
    'XPM',
    'FGA',
    'FGM',
    'FG50',
    'defSack',
    'defI',
    'defSaf',
    'defFum',
    'defBlk',
    'defT',
    'defPtsAgainst',
    'defPassYAgainst',
    'defRushYAgainst',
    'defYdsAgainst'
])

droppedCols = xl1[['season',
'team',
'player',
'pos',
'opponent'
]]

# Encode categorical features
X = pd.get_dummies(X, columns = ['pos', 'posRank'])
X

,week,age,passA_curr,passC_curr,passY_curr,passT_curr,passI_curr,pass2_curr,rushA_curr,rushY_curr,...,posRank_RB2,posRank_RB3,posRank_RB4,posRank_RB5,posRank_TE1,posRank_TE2,posRank_TE3,posRank_TE4,posRank_WR1,posRank_WR2
0,1.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5753,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5754,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5755,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5756,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [284]:
# Insert any missing feature columns
colsFeatures = ['posRank_DF1', 'posRank_PK1', 'posRank_PK2',
       'posRank_PK3', 'posRank_QB1', 'posRank_QB2', 'posRank_QB3',
       'posRank_QB4', 'posRank_RB1', 'posRank_RB2', 'posRank_RB3',
       'posRank_RB4', 'posRank_RB5',
       'posRank_TE1', 'posRank_TE2', 'posRank_TE3', 'posRank_TE4',
       'posRank_TE5', 'posRank_WR1', 'posRank_WR2', 'posRank_WR3',
       ]
colsToAdd = [x for x in colsFeatures if x not in (X.columns)]
X[colsToAdd] = 0
X

,week,age,passA_curr,passC_curr,passY_curr,passT_curr,passI_curr,pass2_curr,rushA_curr,rushY_curr,...,posRank_TE1,posRank_TE2,posRank_TE3,posRank_TE4,posRank_WR1,posRank_WR2,posRank_PK3,posRank_QB4,posRank_TE5,posRank_WR3
0,1.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5753,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5754,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5755,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5756,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [285]:
# Put columns in correct order
X = X[['week',
'age',
'passA_curr',
'passC_curr',
'passY_curr',
'passT_curr',
'passI_curr',
'pass2_curr',
'rushA_curr',
'rushY_curr',
'rushT_curr',
'rush2_curr',
'recC_curr',
'recY_curr',
'recT_curr',
'rec2_curr',
'fum_curr',
'XPA_curr',
'XPM_curr',
'FGA_curr',
'FGM_curr',
'FG50_curr',
'defSack_curr',
'defI_curr',
'defSaf_curr',
'defFum_curr',
'defBlk_curr',
'defT_curr',
'defPtsAgainst_curr',
'defPassYAgainst_curr',
'defRushYAgainst_curr',
'defYdsAgainst_curr',
'gamesPlayed_curr',
'gamesPlayed_prior1',
'passA_prior1',
'passC_prior1',
'passY_prior1',
'passT_prior1',
'passI_prior1',
'pass2_prior1',
'rushA_prior1',
'rushY_prior1',
'rushT_prior1',
'rush2_prior1',
'recC_prior1',
'recY_prior1',
'recT_prior1',
'rec2_prior1',
'fum_prior1',
'XPA_prior1',
'XPM_prior1',
'FGA_prior1',
'FGM_prior1',
'FG50_prior1',
'defSack_prior1',
'defI_prior1',
'defSaf_prior1',
'defFum_prior1',
'defBlk_prior1',
'defT_prior1',
'defPtsAgainst_prior1',
'defPassYAgainst_prior1',
'defRushYAgainst_prior1',
'defYdsAgainst_prior1',
'gamesPlayed_prior2',
'passA_prior2',
'passC_prior2',
'passY_prior2',
'passT_prior2',
'passI_prior2',
'pass2_prior2',
'rushA_prior2',
'rushY_prior2',
'rushT_prior2',
'rush2_prior2',
'recC_prior2',
'recY_prior2',
'recT_prior2',
'rec2_prior2',
'fum_prior2',
'XPA_prior2',
'XPM_prior2',
'FGA_prior2',
'FGM_prior2',
'FG50_prior2',
'defSack_prior2',
'defI_prior2',
'defSaf_prior2',
'defFum_prior2',
'defBlk_prior2',
'defT_prior2',
'defPtsAgainst_prior2',
'defPassYAgainst_prior2',
'defRushYAgainst_prior2',
'defYdsAgainst_prior2',
'defSack_curr_opp',
'defI_curr_opp',
'defSaf_curr_opp',
'defFum_curr_opp',
'defBlk_curr_opp',
'defT_curr_opp',
'defPtsAgainst_curr_opp',
'defPassYAgainst_curr_opp',
'defRushYAgainst_curr_opp',
'defYdsAgainst_curr_opp',
'defSack_prior1_opp',
'defI_prior1_opp',
'defSaf_prior1_opp',
'defFum_prior1_opp',
'defBlk_prior1_opp',
'defT_prior1_opp',
'defPtsAgainst_prior1_opp',
'defPassYAgainst_prior1_opp',
'defRushYAgainst_prior1_opp',
'defYdsAgainst_prior1_opp',
'pos_DF',
'pos_PK',
'pos_QB',
'pos_RB',
'pos_TE',
'pos_WR',
'posRank_DF1',
'posRank_PK1',
'posRank_PK2',
'posRank_PK3',
'posRank_QB1',
'posRank_QB2',
'posRank_QB3',
'posRank_QB4',
'posRank_RB1',
'posRank_RB2',
'posRank_RB3',
'posRank_RB4',
'posRank_RB5',
'posRank_TE1',
'posRank_TE2',
'posRank_TE3',
'posRank_TE4',
'posRank_TE5',
'posRank_WR1',
'posRank_WR2',
'posRank_WR3'
]]


In [265]:
[print(x) for x in X.columns]

week
passA_curr
passC_curr
passY_curr
passT_curr
passI_curr
pass2_curr
rushA_curr
rushY_curr
rushT_curr
rush2_curr
recC_curr
recY_curr
recT_curr
rec2_curr
fum_curr
XPA_curr
XPM_curr
FGA_curr
FGM_curr
FG50_curr
defSack_curr
defI_curr
defSaf_curr
defFum_curr
defBlk_curr
defT_curr
defPtsAgainst_curr
defPassYAgainst_curr
defRushYAgainst_curr
defYdsAgainst_curr
gamesPlayed_curr
gamesPlayed_prior1
passA_prior1
passC_prior1
passY_prior1
passT_prior1
passI_prior1
pass2_prior1
rushA_prior1
rushY_prior1
rushT_prior1
rush2_prior1
recC_prior1
recY_prior1
recT_prior1
rec2_prior1
fum_prior1
XPA_prior1
XPM_prior1
FGA_prior1
FGM_prior1
FG50_prior1
defSack_prior1
defI_prior1
defSaf_prior1
defFum_prior1
defBlk_prior1
defT_prior1
defPtsAgainst_prior1
defPassYAgainst_prior1
defRushYAgainst_prior1
defYdsAgainst_prior1
gamesPlayed_prior2
passA_prior2
passC_prior2
passY_prior2
passT_prior2
passI_prior2
pass2_prior2
rushA_prior2
rushY_prior2
rushT_prior2
rush2_prior2
recC_prior2
recY_prior2
recT_prior2
rec2_p

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [286]:
y_pred = regressor.predict(X)
y_pred = pd.DataFrame(y_pred)
y_pred.columns = ['passA',
'passC',
'passY',
'passT',
'passI',
'pass2',
'rushA',
'rushY',
'rushT',
'rush2',
'recC',
'recY',
'recT',
'rec2',
'fum',
'XPA',
'XPM',
'FGA',
'FGM',
'FG50',
'defSack',
'defI',
'defSaf',
'defFum',
'defBlk',
'defT',
'defPtsAgainst',
'defPassYAgainst',
'defRushYAgainst',
'defYdsAgainst']
y_pred


,passA,passC,passY,passT,passI,pass2,rushA,rushY,rushT,rush2,...,defSack,defI,defSaf,defFum,defBlk,defT,defPtsAgainst,defPassYAgainst,defRushYAgainst,defYdsAgainst
0,35.004912,22.743759,264.323137,1.762313,0.724654,0.075254,3.772725,16.694170,0.175685,0.009461,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,38.820956,25.652958,305.816621,2.143167,0.779901,0.116049,2.557816,9.603997,0.119648,0.005703,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,31.760597,20.088270,224.285302,1.627309,0.779888,0.064540,3.076068,12.805873,0.123055,0.009323,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,39.771342,26.395745,319.319749,2.240328,0.899446,0.090292,2.050753,6.049753,0.077355,0.003160,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,37.222224,24.578135,279.412588,1.867927,0.802697,0.142076,2.511688,8.817103,0.097545,0.003543,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5753,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.490307,0.818255,0.031206,0.542868,0.085484,0.169530,21.645203,214.410397,113.436448,327.846846
5754,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.191972,0.852044,0.030640,0.541500,0.089550,0.181308,24.518064,244.043535,116.544775,360.588310
5755,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.390186,0.789403,0.033574,0.548580,0.087615,0.178080,22.249456,225.218387,111.957440,337.175827
5756,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.057645,0.819189,0.030270,0.494632,0.112381,0.178840,25.364755,245.803327,127.015600,372.818927


In [267]:
# ['passA',0
# 'passC',0
# 'passY',.04
# 'passT',4
# 'passI',-2
# 'pass2',2
# 'rushA',.1
# 'rushY',.1
# 'rushT',6
# 'rush2',2
# 'recC',.25
# 'recY',.1
# 'recT',6
# 'rec2',2
# 'fum',-2
# 'XPA',0
# 'XPM',1
# 'FGA',0
# 'FGM',3
# 'FG50',5
# 'defSack',1
# 'defI',2
# 'defSaf',2
# 'defFum',2
# 'defBlk',1.5
# 'defT',6
# 'defPtsAgainst',0:10, 6:8, 13:7, 17:5, 21:3, 27:2, 34:0, 45:-1, 59:-3, 99:-5
# 'defPassYAgainst',0
# 'defRushYAgainst',0
# 'defYdsAgainst'] 274:5, 324:2, 375:0, 425:-2, 999:-5

multiplier = [
    0,0,.04,4,-2,2,.1,.1,6,2,.25,.1,6,2,-2,0,1,0,3,5,1,2,2,2,1.5,6,0,0,0,0
]

In [287]:
# Calculate FANTASY scores
multiplier = [
    0,0,.04,4,-2,2,.1,.1,6,2,.25,.1,6,2,-2,0,1,0,3,5,1,2,2,2,1.5,6,0,0,0,0,1,1
]
binList_defPts = [-5,0,6,13,17,21,27,34,45,59,99]
binList_defYds = [0,274,324,375,425,999]

ptList_defPts = [10,8,7,5,3,2,0,-1,-3,-5]
ptList_defYds = [5,2,0,-2,-5]

y['defPtsBin'] = pd.cut(y['defPtsAgainst'], bins=binList_defPts, include_lowest=True, labels=ptList_defPts)
y['defYdsBin'] = pd.cut(y['defYdsAgainst'], bins=binList_defYds, include_lowest=True, labels=ptList_defYds)
y_pred['defPtsBin'] = pd.cut(y_pred['defPtsAgainst'], bins=binList_defPts, include_lowest=True, labels=ptList_defPts)
y_pred['defYdsBin'] = pd.cut(y_pred['defYdsAgainst'], bins=binList_defYds, include_lowest=True, labels=ptList_defYds)

a = droppedCols.merge(y, left_index=True, right_index=True)
a_pred = droppedCols.merge(y_pred, left_index=True, right_index=True)

a.loc[a['pos']!='DF', 'defPtsBin'] = 0
a.loc[a['pos']!='DF', 'defYdsBin'] = 0
a_pred.loc[a_pred['pos']!='DF', 'defPtsBin'] = 0
a_pred.loc[a_pred['pos']!='DF', 'defYdsBin'] = 0

a = a.drop(columns=['season','team','player','pos','opponent'])
a_pred = a_pred.drop(columns=['season','team','player','pos','opponent'])


/Users/travisharry/opt/anaconda3/envs/ffpred/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/travisharry/opt/anaconda3/envs/ffpred/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == "":


In [288]:

def multer(row):
    return row.multiply(multiplier)

b = a.apply(multer, axis=1)
b = b.apply(np.sum, axis=1)
b = pd.DataFrame(b, columns=['real'])

c = a_pred.apply(multer, axis=1)
c = c.apply(np.sum, axis=1)
c = pd.DataFrame(c, columns=['pred'])

In [289]:
real_pred = droppedCols.merge(b, left_index=True, right_index=True).merge(c, left_index=True, right_index=True)
real_pred

,season,team,player,pos,opponent,real,pred
0,2021.0,ARI,Kyler Murray,QB,TEN,34.06,18.978149
1,2021.0,KCC,Patrick Mahomes,QB,CLE,33.78,21.050129
2,2021.0,NOS,Jameis Winston,QB,GBP,30.22,15.989795
3,2021.0,TBB,Tom Brady,QB,DAL,27.16,21.052688
4,2021.0,SFO,Jared Goff,QB,DET,29.22,18.618449
...,...,...,...,...,...,...,...
5753,2021.0,MIA,Miami DolphinsMIA,DF,TEN,3.00,8.420369
5754,2021.0,DET,Detroit LionsDET,DF,SEA,-7.00,8.262511
5755,2021.0,DEN,Denver BroncosDEN,DF,LAC,0.00,8.333202
5756,2021.0,MIN,Minnesota VikingsMIN,DF,GBP,-6.00,7.987437


In [290]:
season_totals = real_pred.groupby(['player', 'pos']).sum()
season_totals.drop(columns=['season'], inplace=True)
season_totals

,,real,pred
player,pos,,
A.J. Brown,WR,120.05,125.343796
A.J. Green,WR,113.00,104.996687
AJ Dillon,RB,170.45,131.942193
Aaron Jones,RB,207.10,179.923656
Aaron Rodgers,QB,321.53,279.911289
...,...,...,...
Zach Pascal,WR,65.25,87.480373
Zach Wilson,QB,144.68,154.981433
Zack Moss,RB,95.15,101.565538


In [291]:
season_totals['diff'] = season_totals.real.sub(season_totals.pred).abs()
season_totals

,,real,pred,diff
player,pos,,,
A.J. Brown,WR,120.05,125.343796,5.293796
A.J. Green,WR,113.00,104.996687,8.003313
AJ Dillon,RB,170.45,131.942193,38.507807
Aaron Jones,RB,207.10,179.923656,27.176344
Aaron Rodgers,QB,321.53,279.911289,41.618711
...,...,...,...,...
Zach Pascal,WR,65.25,87.480373,22.230373
Zach Wilson,QB,144.68,154.981433,10.301433
Zack Moss,RB,95.15,101.565538,6.415538


In [292]:
season_totals.sort_values(by='diff', ascending=False).head(20)

,,real,pred,diff
player,pos,,,
Cooper Kupp,WR,309.70,231.456109,78.243891
Dallas CowboysDAL,DF,212.00,134.056187,77.943813
Josh Allen,QB,390.42,314.698808,75.721192
Deebo Samuel,WR,260.35,186.299334,74.050666
Jonathan Taylor,RB,364.55,292.598553,71.951447
James Conner,RB,225.85,162.299129,63.550871
Ja'Marr Chase,WR,241.45,181.605327,59.844673
Jacksonville JaguarsJAX,DF,67.00,126.205163,59.205163
New York JetsNYJ,DF,61.00,120.015534,59.015534


In [293]:
season_totals.sort_values(by='real', ascending=False).head(20)

,,real,pred,diff
player,pos,,,
Josh Allen,QB,390.42,314.698808,75.721192
Jonathan Taylor,RB,364.55,292.598553,71.951447
Justin Herbert,QB,359.44,323.523738,35.916262
Tom Brady,QB,352.50,305.060553,47.439447
Patrick Mahomes,QB,341.16,319.779374,21.380626
Jalen Hurts,QB,326.06,267.938085,58.121915
Aaron Rodgers,QB,321.53,279.911289,41.618711
Joe Burrow,QB,318.24,287.213163,31.026837
Matthew Stafford,QB,315.42,271.009971,44.410029


In [294]:
season_totals.sort_values(by='pred', ascending=False).head(20)

,,real,pred,diff
player,pos,,,
Justin Herbert,QB,359.44,323.523738,35.916262
Patrick Mahomes,QB,341.16,319.779374,21.380626
Josh Allen,QB,390.42,314.698808,75.721192
Tom Brady,QB,352.50,305.060553,47.439447
Jonathan Taylor,RB,364.55,292.598553,71.951447
Derek Carr,QB,245.52,290.809199,45.289199
Joe Burrow,QB,318.24,287.213163,31.026837
Aaron Rodgers,QB,321.53,279.911289,41.618711
Dak Prescott,QB,293.56,279.541387,14.018613


In [ ]:
# Create master player db
# Get 2022 opponents
    # Merge opponents to coming stats
# Deal with currentstats throwing things out of whack before season starts
# Add in Punt- and Kick-Return scores